In [12]:
pip install fastparquet


Note: you may need to restart the kernel to use updated packages.


In [13]:
import pandas as pd
from pathlib import Path
from typing import Union

# Pastas
base_folder = Path("MABe-mouse-behavior-detection")
tracking_folder = base_folder / "train_tracking"
annotation_folder = base_folder / "train_annotation"
df_meta = pd.read_csv(base_folder / "train.csv")

# Nova pasta de saída
output_folder = base_folder / "processed_videos_final_fixed" # Nova pasta para garantir limpeza
output_folder.mkdir(exist_ok=True)

def load_parquet(file_path: Path) -> pd.DataFrame:
    return pd.read_parquet(file_path, engine="fastparquet")

def extract_video_id(file_path: Path) -> int:
    return int(file_path.stem)

def pivot_tracking(df_track: pd.DataFrame, n_mice: int = 4) -> pd.DataFrame:
    """
    Transforma tracking em wide com 1 linha por frame, garantindo todas as colunas para até n_mice.
    """
    df = df_track.rename(columns={"video_frame": "frame"})
    # 🚨 Usamos 'pivot' (não 'pivot_table') para evitar problemas de agregação/perda
    df["col_x"] = "mouse" + df["mouse_id"].astype(str) + "_" + df["bodypart"] + "_x"
    df["col_y"] = "mouse" + df["mouse_id"].astype(str) + "_" + df["bodypart"] + "_y"

    # Tentamos o pivot. Se falhar, é porque o 'df_track' original tinha duplicatas por frame/mouse/bodypart
    try:
        wide_x = df.pivot(index="frame", columns="col_x", values="x")
        wide_y = df.pivot(index="frame", columns="col_y", values="y")
    except ValueError as e:
        # Se houver duplicatas no df_track_raw (raro, mas possível), usamos pivot_table com first.
        print(f"⚠️ Aviso: Duplicatas encontradas no tracking. Usando pivot_table com 'first'. Erro: {e}")
        wide_x = df.pivot_table(index="frame", columns="col_x", values="x", aggfunc="first")
        wide_y = df.pivot_table(index="frame", columns="col_y", values="y", aggfunc="first")

    wide = pd.concat([wide_x, wide_y], axis=1).reset_index()
    
    # 🔹 Adiciona colunas faltantes para garantir os 4 ratos
    bodyparts = df["bodypart"].unique()
    
    for m in range(1, n_mice + 1):
        for bp in bodyparts:
            for axis in ["x", "y"]:
                col = f"mouse{m}_{bp}_{axis}"
                if col not in wide.columns:
                    wide[col] = pd.NA 

    tracking_cols = ["frame"] + sorted([c for c in wide.columns if c != "frame"])
    return wide[tracking_cols]

def expand_annotations(df_ann: pd.DataFrame) -> pd.DataFrame:
    """Expande intervalos de anotação em uma linha por frame, agregando múltiplos comportamentos em lista."""
    if df_ann.empty:
        return pd.DataFrame(columns=["frame", "behavior"])

    expanded = []
    for _, row in df_ann.iterrows():
        for f in range(int(row["start_frame"]), int(row["stop_frame"]) + 1):
            expanded.append({"frame": f, "behavior": row["action"]})

    df_exp = pd.DataFrame(expanded)

    # 🚨 Agrupa múltiplos comportamentos por frame para EVITAR DUPLICAÇÃO DE LINHA
    return (
        df_exp.groupby("frame")["behavior"]
        .apply(list) # Agrega em uma lista de strings
        .reset_index()
    )

# Itera sobre laboratórios
for lab_folder in tracking_folder.iterdir():
    if not lab_folder.is_dir():
        continue

    print(f"📂 Processando laboratório {lab_folder.name}...")

    tracking_files = list(lab_folder.glob("*.parquet"))
    annotation_lab_folder = annotation_folder / lab_folder.name
    annotation_files = list(annotation_lab_folder.glob("*.parquet"))

    videoid_to_annotation = {extract_video_id(f): load_parquet(f) for f in annotation_files}

    for track_file in tracking_files:
        vid = extract_video_id(track_file)
        df_track_raw = load_parquet(track_file)
        
        # 1. Pivotar tracking (1 linha = 1 frame, com todos os 4 ratos garantidos)
        df_track = pivot_tracking(df_track_raw, n_mice=4)

        # 2. Expandir annotations (1 linha = 1 frame, comportamentos em lista)
        df_ann = videoid_to_annotation.get(
            vid, pd.DataFrame(columns=["start_frame", "stop_frame", "action"])
        )
        df_ann_expanded = expand_annotations(df_ann)

        # 3. Merge tracking + annotations
        # O merge aqui é 1:1, pois df_track e df_ann_expanded têm no máximo 1 linha por frame.
        df_merged = pd.merge(df_track, df_ann_expanded, on="frame", how="left")

        # 4. Adicionar metadata
        meta_row = df_meta[df_meta["video_id"] == vid]
        if not meta_row.empty:
            # Garante que os metadados sejam adicionados como colunas constantes
            meta_dict = meta_row.iloc[0].to_dict()
            for col, val in meta_dict.items():
                if col not in df_merged.columns:
                    df_merged[col] = val

        # 5. GARANTIR UMA LINHA PARA CADA FRAME (0 até MAX)
        if not df_merged.empty:
            # Se o tracking existir, usa o range do frame min ao frame max
            all_frames = pd.DataFrame(
                {"frame": range(int(df_merged["frame"].min()), int(df_merged["frame"].max()) + 1)}
            )
            df_full = all_frames.merge(df_merged, on="frame", how="left")
        else:
            # Caso o vídeo seja vazio (improvável)
            df_full = df_merged.copy() 

        # Salva arquivo processado individual
        out_path = output_folder / f"{vid}_processed.parquet"
        df_full.to_parquet(out_path, index=False)
        print(f"✅ Vídeo {vid} processado e salvo em {out_path} (Linhas: {len(df_full)})")

📂 Processando laboratório AdaptableSnail...
✅ Vídeo 1212811043 processado e salvo em MABe-mouse-behavior-detection\processed_videos_final_fixed\1212811043_processed.parquet (Linhas: 89976)
✅ Vídeo 1260392287 processado e salvo em MABe-mouse-behavior-detection\processed_videos_final_fixed\1260392287_processed.parquet (Linhas: 53918)
✅ Vídeo 1351098077 processado e salvo em MABe-mouse-behavior-detection\processed_videos_final_fixed\1351098077_processed.parquet (Linhas: 81780)
✅ Vídeo 1408652858 processado e salvo em MABe-mouse-behavior-detection\processed_videos_final_fixed\1408652858_processed.parquet (Linhas: 18432)
✅ Vídeo 143861384 processado e salvo em MABe-mouse-behavior-detection\processed_videos_final_fixed\143861384_processed.parquet (Linhas: 89976)
✅ Vídeo 1596473327 processado e salvo em MABe-mouse-behavior-detection\processed_videos_final_fixed\1596473327_processed.parquet (Linhas: 89976)
✅ Vídeo 1643942986 processado e salvo em MABe-mouse-behavior-detection\processed_videos_

In [14]:
import pandas as pd
from pathlib import Path

# Caminho para a pasta de saída
output_folder = Path("MABe-mouse-behavior-detection") / "processed_videos_final_fixed"

# Escolher um arquivo processado (ex: o primeiro da pasta)
parquet_files = list(output_folder.glob("*.parquet"))
print("Arquivos disponíveis:", parquet_files[:5])  # mostra alguns nomes

# Carregar o primeiro para visualizar
df = pd.read_parquet(parquet_files[1], engine="fastparquet")

# Mostrar infos básicas
print("\n📊 Shape:", df.shape)
print(df.head(10))  # mostra as primeiras linhas


Arquivos disponíveis: [WindowsPath('MABe-mouse-behavior-detection/processed_videos_final_fixed/1000217804_processed.parquet'), WindowsPath('MABe-mouse-behavior-detection/processed_videos_final_fixed/1000285113_processed.parquet'), WindowsPath('MABe-mouse-behavior-detection/processed_videos_final_fixed/1000942438_processed.parquet'), WindowsPath('MABe-mouse-behavior-detection/processed_videos_final_fixed/1001006733_processed.parquet'), WindowsPath('MABe-mouse-behavior-detection/processed_videos_final_fixed/1001081757_processed.parquet')]

📊 Shape: (1800, 136)
   frame  mouse1_body_center_x  mouse1_body_center_y  mouse1_ear_left_x  \
0      0                 702.0                 714.0              714.0   
1      1                 701.0                 715.0              710.0   
2      2                 700.0                 715.0              705.0   
3      3                 700.0                 716.0              704.0   
4      4                 700.0                 716.0        

In [15]:
df

,frame,mouse1_body_center_x,mouse1_body_center_y,mouse1_ear_left_x,mouse1_ear_left_y,mouse1_ear_right_x,mouse1_ear_right_y,mouse1_forepaw_left_x,mouse1_forepaw_left_y,mouse1_forepaw_right_x,...,pix_per_cm_approx,video_width_pix,video_height_pix,arena_width_cm,arena_height_cm,arena_shape,arena_type,body_parts_tracked,behaviors_labeled,tracking_method
0,0,702.0,714.0,714.0,192.0,650.0,135.0,710.0,172.0,667.0,...,11.7,800,800,52.0,52.0,square,neutral,"[""body_center"", ""ear_left"", ""ear_right"", ""fore...",NaN,custom HRnet
1,1,701.0,715.0,710.0,194.0,650.0,135.0,711.0,175.0,668.0,...,11.7,800,800,52.0,52.0,square,neutral,"[""body_center"", ""ear_left"", ""ear_right"", ""fore...",NaN,custom HRnet
2,2,700.0,715.0,705.0,198.0,651.0,135.0,712.0,178.0,667.0,...,11.7,800,800,52.0,52.0,square,neutral,"[""body_center"", ""ear_left"", ""ear_right"", ""fore...",NaN,custom HRnet
3,3,700.0,716.0,704.0,200.0,651.0,135.0,713.0,179.0,666.0,...,11.7,800,800,52.0,52.0,square,neutral,"[""body_center"", ""ear_left"", ""ear_right"", ""fore...",NaN,custom HRnet
4,4,700.0,716.0,705.0,201.0,651.0,135.0,713.0,180.0,666.0,...,11.7,800,800,52.0,52.0,square,neutral,"[""body_center"", ""ear_left"", ""ear_right"", ""fore...",NaN,custom HRnet
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1795,1795,672.0,178.0,604.0,158.0,0.0,0.0,623.0,157.0,0.0,...,11.7,800,800,52.0,52.0,square,neutral,"[""body_center"", ""ear_left"", ""ear_right"", ""fore...",NaN,custom HRnet
1796,1796,672.0,178.0,604.0,157.0,0.0,0.0,623.0,157.0,0.0,...,11.7,800,800,52.0,52.0,square,neutral,"[""body_center"", ""ear_left"", ""ear_right"", ""fore...",NaN,custom HRnet
1797,1797,672.0,178.0,604.0,155.0,0.0,0.0,623.0,158.0,0.0,...,11.7,800,800,52.0,52.0,square,neutral,"[""body_center"", ""ear_left"", ""ear_right"", ""fore...",NaN,custom HRnet
1798,1798,672.0,179.0,604.0,154.0,0.0,0.0,623.0,158.0,0.0,...,11.7,800,800,52.0,52.0,square,neutral,"[""body_center"", ""ear_left"", ""ear_right"", ""fore...",NaN,custom HRnet


In [16]:
df['frame'].value_counts()

frame
0       1
1       1
2       1
3       1
4       1
       ..
1795    1
1796    1
1797    1
1798    1
1799    1
Name: count, Length: 1800, dtype: int64

In [17]:
df.columns

Index(['frame', 'mouse1_body_center_x', 'mouse1_body_center_y',
       'mouse1_ear_left_x', 'mouse1_ear_left_y', 'mouse1_ear_right_x',
       'mouse1_ear_right_y', 'mouse1_forepaw_left_x', 'mouse1_forepaw_left_y',
       'mouse1_forepaw_right_x',
       ...
       'pix_per_cm_approx', 'video_width_pix', 'video_height_pix',
       'arena_width_cm', 'arena_height_cm', 'arena_shape', 'arena_type',
       'body_parts_tracked', 'behaviors_labeled', 'tracking_method'],
      dtype='object', length=136)

In [18]:
df.columns[:50]

Index(['frame', 'mouse1_body_center_x', 'mouse1_body_center_y',
       'mouse1_ear_left_x', 'mouse1_ear_left_y', 'mouse1_ear_right_x',
       'mouse1_ear_right_y', 'mouse1_forepaw_left_x', 'mouse1_forepaw_left_y',
       'mouse1_forepaw_right_x', 'mouse1_forepaw_right_y',
       'mouse1_hindpaw_left_x', 'mouse1_hindpaw_left_y',
       'mouse1_hindpaw_right_x', 'mouse1_hindpaw_right_y', 'mouse1_neck_x',
       'mouse1_neck_y', 'mouse1_nose_x', 'mouse1_nose_y', 'mouse1_tail_base_x',
       'mouse1_tail_base_y', 'mouse1_tail_midpoint_x',
       'mouse1_tail_midpoint_y', 'mouse1_tail_tip_x', 'mouse1_tail_tip_y',
       'mouse2_body_center_x', 'mouse2_body_center_y', 'mouse2_ear_left_x',
       'mouse2_ear_left_y', 'mouse2_ear_right_x', 'mouse2_ear_right_y',
       'mouse2_forepaw_left_x', 'mouse2_forepaw_left_y',
       'mouse2_forepaw_right_x', 'mouse2_forepaw_right_y',
       'mouse2_hindpaw_left_x', 'mouse2_hindpaw_left_y',
       'mouse2_hindpaw_right_x', 'mouse2_hindpaw_right_y', 'mou

In [19]:
df.columns[50:100]

Index(['mouse3_body_center_y', 'mouse3_ear_left_x', 'mouse3_ear_left_y',
       'mouse3_ear_right_x', 'mouse3_ear_right_y', 'mouse3_forepaw_left_x',
       'mouse3_forepaw_left_y', 'mouse3_forepaw_right_x',
       'mouse3_forepaw_right_y', 'mouse3_hindpaw_left_x',
       'mouse3_hindpaw_left_y', 'mouse3_hindpaw_right_x',
       'mouse3_hindpaw_right_y', 'mouse3_neck_x', 'mouse3_neck_y',
       'mouse3_nose_x', 'mouse3_nose_y', 'mouse3_tail_base_x',
       'mouse3_tail_base_y', 'mouse3_tail_midpoint_x',
       'mouse3_tail_midpoint_y', 'mouse3_tail_tip_x', 'mouse3_tail_tip_y',
       'mouse4_body_center_x', 'mouse4_body_center_y', 'mouse4_ear_left_x',
       'mouse4_ear_left_y', 'mouse4_ear_right_x', 'mouse4_ear_right_y',
       'mouse4_forepaw_left_x', 'mouse4_forepaw_left_y',
       'mouse4_forepaw_right_x', 'mouse4_forepaw_right_y',
       'mouse4_hindpaw_left_x', 'mouse4_hindpaw_left_y',
       'mouse4_hindpaw_right_x', 'mouse4_hindpaw_right_y', 'mouse4_neck_x',
       'mouse4_neck_

In [20]:
df.columns[90:130]

Index(['mouse4_nose_y', 'mouse4_tail_base_x', 'mouse4_tail_base_y',
       'mouse4_tail_midpoint_x', 'mouse4_tail_midpoint_y', 'mouse4_tail_tip_x',
       'mouse4_tail_tip_y', 'behavior', 'lab_id', 'video_id', 'mouse1_strain',
       'mouse1_color', 'mouse1_sex', 'mouse1_id', 'mouse1_age',
       'mouse1_condition', 'mouse2_strain', 'mouse2_color', 'mouse2_sex',
       'mouse2_id', 'mouse2_age', 'mouse2_condition', 'mouse3_strain',
       'mouse3_color', 'mouse3_sex', 'mouse3_id', 'mouse3_age',
       'mouse3_condition', 'mouse4_strain', 'mouse4_color', 'mouse4_sex',
       'mouse4_id', 'mouse4_age', 'mouse4_condition', 'frames_per_second',
       'video_duration_sec', 'pix_per_cm_approx', 'video_width_pix',
       'video_height_pix', 'arena_width_cm'],
      dtype='object')

In [21]:
df.columns[120:]

Index(['mouse4_sex', 'mouse4_id', 'mouse4_age', 'mouse4_condition',
       'frames_per_second', 'video_duration_sec', 'pix_per_cm_approx',
       'video_width_pix', 'video_height_pix', 'arena_width_cm',
       'arena_height_cm', 'arena_shape', 'arena_type', 'body_parts_tracked',
       'behaviors_labeled', 'tracking_method'],
      dtype='object')

In [22]:
import pandas as pd
from pathlib import Path
from typing import Union

# Pasta onde os arquivos limpos estão
INPUT_FOLDER = Path("MABe-mouse-behavior-detection") / "processed_videos_final_fixed"

def merge_parquets(folder_path: Union[str, Path], max_files: int = 0) -> pd.DataFrame:
    """Carrega e combina todos os arquivos .parquet em uma pasta."""
    folder = Path(folder_path)
    parquet_files = list(folder.glob("*.parquet"))
    
    if not parquet_files:
        print(f"⚠️ Aviso: Nenhuma arquivo .parquet encontrado em {folder_path}. Retornando DataFrame vazio.")
        return pd.DataFrame()
    
    if max_files > 0:
        parquet_files = parquet_files[:max_files]
        print(f"Processando os primeiros {len(parquet_files)} arquivos...")
    else:
        print(f"Processando um total de {len(parquet_files)} arquivos...")
        
    all_data = []
    
    for file in parquet_files:
        try:
            df = pd.read_parquet(file, engine='fastparquet')
            all_data.append(df)
        except Exception as e:
            print(f"❌ Erro ao ler o arquivo {file.name}: {e}")
            
    if all_data:
        merged_df = pd.concat(all_data, ignore_index=True)
        print(f"✅ Concatenação concluída! DataFrame final com {len(merged_df)} linhas.")
        return merged_df
    else:
        return pd.DataFrame()

# ==============================================================================
# EXECUÇÃO DA CONCATENAÇÃO
# ==============================================================================

print(f"\n--- Iniciando a Combinação Final dos {INPUT_FOLDER.name} ---")

# Carrega e combina TODOS os arquivos da pasta
df_final_dataset = merge_parquets(INPUT_FOLDER, max_files=30)

# Se o merge for bem-sucedido, faz a verificação final de unicidade
if not df_final_dataset.empty:
    print("\n--- Verificação de Duplicação Global ---")
    
    # Cria um ID ÚNICO GLOBAL (video_id + frame)
    df_final_dataset['unique_frame_id'] = (
        df_final_dataset['video_id'].astype(str) + 
        '_' + 
        df_final_dataset['frame'].astype(str)
    )

    # Conta as duplicatas no ID GLOBAL
    duplicates = df_final_dataset['unique_frame_id'].value_counts()
    duplicates = duplicates[duplicates > 1]
    
    print(f"Total de linhas no dataset combinado: {len(df_final_dataset)}")
    
    if duplicates.empty:
        print("✅ SUCESSO: Nenhuma duplicação de linha (video_id + frame) encontrada no dataset combinado.")
    else:
        print(f"❌ ERRO GRAVE: {len(duplicates)} linhas ainda estão duplicadas. Algo no source data está incorreto.")
        print(duplicates.head())


--- Iniciando a Combinação Final dos processed_videos_final_fixed ---
Processando os primeiros 30 arquivos...
✅ Concatenação concluída! DataFrame final com 70591 linhas.

--- Verificação de Duplicação Global ---
Total de linhas no dataset combinado: 70591
✅ SUCESSO: Nenhuma duplicação de linha (video_id + frame) encontrada no dataset combinado.


C:\Users\Lucas\AppData\Local\Temp\ipykernel_3096\2405624018.py:33: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  merged_df = pd.concat(all_data, ignore_index=True)


In [23]:
df_final_dataset

,frame,mouse1_body_center_x,mouse1_body_center_y,mouse1_ear_left_x,mouse1_ear_left_y,mouse1_ear_right_x,mouse1_ear_right_y,mouse1_forepaw_left_x,mouse1_forepaw_left_y,mouse1_forepaw_right_x,...,mouse2_hip_right_y,mouse3_hip_left_x,mouse3_hip_left_y,mouse3_hip_right_x,mouse3_hip_right_y,mouse4_hip_left_x,mouse4_hip_left_y,mouse4_hip_right_x,mouse4_hip_right_y,unique_frame_id
0,0,334.0,141.0,129.000000,205.000000,192.000000,134.000000,150.0,186.0,154.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1000217804_0
1,1,334.0,142.0,129.000000,204.000000,192.000000,139.000000,151.0,185.0,153.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1000217804_1
2,2,334.0,142.0,128.000000,203.000000,0.000000,0.000000,151.0,185.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1000217804_2
3,3,334.0,142.0,128.000000,203.000000,0.000000,0.000000,151.0,186.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1000217804_3
4,4,334.0,142.0,128.000000,203.000000,0.000000,0.000000,151.0,187.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1000217804_4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
70586,18386,NaN,NaN,793.293091,437.135162,760.293091,501.135162,NaN,NaN,NaN,...,402.403473,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1006083669_18386
70587,18387,NaN,NaN,793.734314,437.645599,760.734314,499.645599,NaN,NaN,NaN,...,406.315277,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1006083669_18387
70588,18388,NaN,NaN,793.591553,437.792938,761.591553,501.792938,NaN,NaN,NaN,...,406.286530,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1006083669_18388
70589,18389,NaN,NaN,793.414734,437.458466,760.414734,501.458466,NaN,NaN,NaN,...,405.579132,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1006083669_18389


In [24]:
df_final_dataset.columns[:55]

Index(['frame', 'mouse1_body_center_x', 'mouse1_body_center_y',
       'mouse1_ear_left_x', 'mouse1_ear_left_y', 'mouse1_ear_right_x',
       'mouse1_ear_right_y', 'mouse1_forepaw_left_x', 'mouse1_forepaw_left_y',
       'mouse1_forepaw_right_x', 'mouse1_forepaw_right_y',
       'mouse1_hindpaw_left_x', 'mouse1_hindpaw_left_y',
       'mouse1_hindpaw_right_x', 'mouse1_hindpaw_right_y', 'mouse1_neck_x',
       'mouse1_neck_y', 'mouse1_nose_x', 'mouse1_nose_y', 'mouse1_tail_base_x',
       'mouse1_tail_base_y', 'mouse1_tail_midpoint_x',
       'mouse1_tail_midpoint_y', 'mouse1_tail_tip_x', 'mouse1_tail_tip_y',
       'mouse2_body_center_x', 'mouse2_body_center_y', 'mouse2_ear_left_x',
       'mouse2_ear_left_y', 'mouse2_ear_right_x', 'mouse2_ear_right_y',
       'mouse2_forepaw_left_x', 'mouse2_forepaw_left_y',
       'mouse2_forepaw_right_x', 'mouse2_forepaw_right_y',
       'mouse2_hindpaw_left_x', 'mouse2_hindpaw_left_y',
       'mouse2_hindpaw_right_x', 'mouse2_hindpaw_right_y', 'mou

In [25]:
df_final_dataset.columns[55:120]

Index(['mouse3_forepaw_left_x', 'mouse3_forepaw_left_y',
       'mouse3_forepaw_right_x', 'mouse3_forepaw_right_y',
       'mouse3_hindpaw_left_x', 'mouse3_hindpaw_left_y',
       'mouse3_hindpaw_right_x', 'mouse3_hindpaw_right_y', 'mouse3_neck_x',
       'mouse3_neck_y', 'mouse3_nose_x', 'mouse3_nose_y', 'mouse3_tail_base_x',
       'mouse3_tail_base_y', 'mouse3_tail_midpoint_x',
       'mouse3_tail_midpoint_y', 'mouse3_tail_tip_x', 'mouse3_tail_tip_y',
       'mouse4_body_center_x', 'mouse4_body_center_y', 'mouse4_ear_left_x',
       'mouse4_ear_left_y', 'mouse4_ear_right_x', 'mouse4_ear_right_y',
       'mouse4_forepaw_left_x', 'mouse4_forepaw_left_y',
       'mouse4_forepaw_right_x', 'mouse4_forepaw_right_y',
       'mouse4_hindpaw_left_x', 'mouse4_hindpaw_left_y',
       'mouse4_hindpaw_right_x', 'mouse4_hindpaw_right_y', 'mouse4_neck_x',
       'mouse4_neck_y', 'mouse4_nose_x', 'mouse4_nose_y', 'mouse4_tail_base_x',
       'mouse4_tail_base_y', 'mouse4_tail_midpoint_x',
       'm

In [26]:
df_final_dataset.columns[120:]

Index(['mouse4_sex', 'mouse4_id', 'mouse4_age', 'mouse4_condition',
       'frames_per_second', 'video_duration_sec', 'pix_per_cm_approx',
       'video_width_pix', 'video_height_pix', 'arena_width_cm',
       'arena_height_cm', 'arena_shape', 'arena_type', 'body_parts_tracked',
       'behaviors_labeled', 'tracking_method', 'mouse1_hip_left_x',
       'mouse1_hip_left_y', 'mouse1_hip_right_x', 'mouse1_hip_right_y',
       'mouse2_hip_left_x', 'mouse2_hip_left_y', 'mouse2_hip_right_x',
       'mouse2_hip_right_y', 'mouse3_hip_left_x', 'mouse3_hip_left_y',
       'mouse3_hip_right_x', 'mouse3_hip_right_y', 'mouse4_hip_left_x',
       'mouse4_hip_left_y', 'mouse4_hip_right_x', 'mouse4_hip_right_y',
       'unique_frame_id'],
      dtype='object')

In [29]:
import pandas as pd
import numpy as np

# A variável df_final_dataset deve estar carregada aqui, resultante do seu merge_parquets.

# --------------------------------------------------------------------------------
# 1. Identificação de Colunas
# --------------------------------------------------------------------------------

# Encontra TODAS as colunas de coordenadas (_x, _y)
coord_cols = [col for col in df_final_dataset.columns 
              if (col.endswith('_x') or col.endswith('_y')) and col.startswith('mouse')]

# --------------------------------------------------------------------------------
# 2. Tratamento de NaN e 0.0 (Interpolação) - CORRIGIDO com .transform()
# --------------------------------------------------------------------------------

print("Iniciando tratamento de NaNs e 0.0...")

# 2a. Padroniza 0.0 como NaN nas coordenadas (Assume que 0.0 significa 'não detectado')
for col in coord_cols:
    df_final_dataset[col] = df_final_dataset[col].replace(0.0, np.nan)

# 2b. Interpolação Linear (AGORA USANDO .transform() para garantir alinhamento de índice)
for col in coord_cols:
    df_final_dataset[col] = df_final_dataset.groupby('video_id')[col].transform(
        lambda x: x.interpolate(method='linear', limit=10, limit_direction='both')
    )

print("✅ Interpolação de tracking concluída.")

# --------------------------------------------------------------------------------
# 3. Normalização: Pixels para Centímetros (CM)
# --------------------------------------------------------------------------------

print("Iniciando conversão de Pixels para CM...")

for col in coord_cols:
    col_cm = col.replace('_x', '_cm_x').replace('_y', '_cm_y')
    # Divide a coordenada (em pixels) pelo fator de conversão (pix_per_cm_approx)
    df_final_dataset[col_cm] = df_final_dataset[col] / df_final_dataset['pix_per_cm_approx']

tracking_cols_cm = [col for col in df_final_dataset.columns if col.endswith('_cm_x') or col.endswith('_cm_y')]


# --------------------------------------------------------------------------------
# 4. Criação de Features de Movimento (Velocidade)
# --------------------------------------------------------------------------------

print("Iniciando cálculo de Velocidade...")

for m in range(1, 5):
    center_x_cm = f'mouse{m}_body_center_cm_x'
    center_y_cm = f'mouse{m}_body_center_cm_y'
    
    # Calcula a diferença de X (delta X) e Y (delta Y) entre frames
    df_final_dataset[f'mouse{m}_delta_x'] = df_final_dataset.groupby('video_id')[center_x_cm].diff()
    df_final_dataset[f'mouse{m}_delta_y'] = df_final_dataset.groupby('video_id')[center_y_cm].diff()
    
    # Calcula a velocidade (distância euclidiana da mudança: sqrt(dx² + dy²))
    df_final_dataset[f'mouse{m}_speed_cm_per_frame'] = np.sqrt(
        df_final_dataset[f'mouse{m}_delta_x']**2 + df_final_dataset[f'mouse{m}_delta_y']**2
    )

# --------------------------------------------------------------------------------
# 5. Criação de Features de Interação (Distância entre Ratos)
# --------------------------------------------------------------------------------

print("Iniciando cálculo de Distância Social...")

# Distância entre Mouse 1 e Mouse 2 (usando o centro do corpo)
center1_x = 'mouse1_body_center_cm_x'
center1_y = 'mouse1_body_center_cm_y'
center2_x = 'mouse2_body_center_cm_x'
center2_y = 'mouse2_body_center_cm_y'

df_final_dataset['dist_m1_m2_cm'] = np.sqrt(
    (df_final_dataset[center1_x] - df_final_dataset[center2_x])**2 +
    (df_final_dataset[center1_y] - df_final_dataset[center2_y])**2
)

print("✅ Feature Engineering concluído!")
print(f"Novo Shape do Dataset: {df_final_dataset.shape}")


Iniciando tratamento de NaNs e 0.0...
✅ Interpolação de tracking concluída.
Iniciando conversão de Pixels para CM...
Iniciando cálculo de Velocidade...
Iniciando cálculo de Distância Social...
✅ Feature Engineering concluído!
Novo Shape do Dataset: (70591, 278)


C:\Users\Lucas\AppData\Local\Temp\ipykernel_3096\1136634471.py:41: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_final_dataset[col_cm] = df_final_dataset[col] / df_final_dataset['pix_per_cm_approx']
C:\Users\Lucas\AppData\Local\Temp\ipykernel_3096\1136634471.py:41: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_final_dataset[col_cm] = df_final_dataset[col] / df_final_dataset['pix_per_cm_approx']
C:\Users\Lucas\AppData\Local\Temp\ipykernel_3096\1136634471.py:41: PerformanceWarning: DataFrame is highly fragmented.  This is us

In [30]:
df_final_dataset

,frame,mouse1_body_center_x,mouse1_body_center_y,mouse1_ear_left_x,mouse1_ear_left_y,mouse1_ear_right_x,mouse1_ear_right_y,mouse1_forepaw_left_x,mouse1_forepaw_left_y,mouse1_forepaw_right_x,...,mouse2_delta_x,mouse2_delta_y,mouse2_speed_cm_per_frame,mouse3_delta_x,mouse3_delta_y,mouse3_speed_cm_per_frame,mouse4_delta_x,mouse4_delta_y,mouse4_speed_cm_per_frame,dist_m1_m2_cm
0,0,334.0,141.0,129.000000,205.000000,192.000000,134.000000,150.0,186.0,154.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.939976
1,1,334.0,142.0,129.000000,204.000000,192.000000,139.000000,151.0,185.0,153.0,...,0.0,0.0,0.0,0.00000,0.00000,0.000000,NaN,NaN,NaN,3.946460
2,2,334.0,142.0,128.000000,203.000000,192.000000,139.000000,151.0,185.0,151.5,...,0.0,0.0,0.0,-0.08547,-0.08547,0.120873,NaN,NaN,NaN,3.946460
3,3,334.0,142.0,128.000000,203.000000,192.000000,139.000000,151.0,186.0,150.0,...,0.0,0.0,0.0,0.00000,0.00000,0.000000,NaN,NaN,NaN,3.946460
4,4,334.0,142.0,128.000000,203.000000,192.000000,139.000000,151.0,187.0,148.5,...,0.0,0.0,0.0,0.00000,0.00000,0.000000,NaN,NaN,NaN,3.946460
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
70586,18386,NaN,NaN,793.293091,437.135162,760.293091,501.135162,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
70587,18387,NaN,NaN,793.734314,437.645599,760.734314,499.645599,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
70588,18388,NaN,NaN,793.591553,437.792938,761.591553,501.792938,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
70589,18389,NaN,NaN,793.414734,437.458466,760.414734,501.458466,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
